In [1]:
#importing libraries
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#creating big dataset, combining all the players and their scores into one csv file
stats =  {}
x = 18
for y in range (1999, 2022):
   
    if y == 2021:
        x = 19
    for i in range(1,x):
        path_name = f"/Users/composerinmaking/Downloads/data-master/weekly/{y}/Week{i}.csv"
        
        data = pd.read_csv(path_name)
    
        for i in data.index:
            Player = data['Player'][i]
            SFP =  data['StandardFantasyPoints'][i]
            PPR = data['PPRFantasyPoints'][i]
            HPPR = data['HalfPPRFantasyPoints'][i]
            if Player not in stats: 
                stats[Player] = [[SFP], [PPR], [HPPR]]
            
            else: 
                stats[Player][0].append(SFP)
                stats[Player][1].append(PPR)
                stats[Player][2].append(HPPR)
    print(y, " finished")
stats1 = pd.DataFrame.from_dict(stats, orient = "index", columns=['SFP', 'PPR', 'HPPR'])

stats1.loc['Amari Cooper', 'SFP'][0:5]
stats1.loc['Amari Cooper', 'PPR'][0:5]
stats1.loc['Amari Cooper', 'HPPR'][0:5]
m , n= stats1.shape

df = pd.DataFrame(columns = ['InputSFP', 'OutputSFP', 'InputPPR', 'OutputPPR', 'InputHPPR', 'OutputHPPR'])

for i in range(0, m): 
    weeks_played = len(stats1.iloc[i]['SFP'])
    #print(weeks_played)
    for x in range(weeks_played - 5 -1): 
        input_data = stats1.iloc[i]['SFP'][x:x+5]
        output_data = stats1.iloc[i]['SFP'][x+5]
        input_data1 = stats1.iloc[i]['PPR'][x:x+5]
        output_data1 = stats1.iloc[i]['PPR'][x+5]
        input_data2 = stats1.iloc[i]['HPPR'][x:x+5]
        output_data2 = stats1.iloc[i]['HPPR'][x+5]
        df.loc[len(df.index)] = [input_data, output_data, input_data1, output_data1, input_data2, output_data2]
    print(i, " finished")    
        
    
        #print (output_data)
        #print (input_data)
#df.head()
df.to_csv("football_data.csv",index = False)


In [ ]:
#data cleaning
fbdata = pd.read_csv('football_data.csv')

for i in fbdata.index:
    x = fbdata['InputSFP'][i]
    z = fbdata['InputPPR'][i]
    v = fbdata['InputHPPR'][i]
    y = x.strip('][').split(', ')
    y = [float(i) for i in y]
    a = z.strip('][').split(', ')
    a = [float(i) for i in a]
    b = v.strip('][').split(', ')
    b = [float(i) for i in b]

    #fbdata['InputSFP'][i] = y
    #fbdata['InputPPR'][i] = a
    #bdata['InputHPPR'][i] = b
    
    fbdata.at[i, 'InputSFP'] = y 
    fbdata.at[i, 'InputPPR'] = a 
    fbdata.at[i, 'InputHPPR'] = b 
    if i % 100 == 0: 
        print(i)
        
print(fbdata)


In [ ]:
fbdata.head()

In [ ]:
fbdata = fbdata.sample(frac=1)

In [ ]:
#splitting data into train, test, val
train = fbdata[:int(len(fbdata)* .7)]
train.to_csv('training_footballData.csv')
valid = fbdata[int(len(fbdata)* .7): int(len(fbdata)* .9)]
valid.to_csv('validation_footballData.csv')
test = fbdata[int(len(fbdata)* .9):]
test.to_csv('testing_footballData.csv')
yy = pd.read_csv('training_footballData.csv')


In [ ]:
#more cleaning
class FantasyFootball(Dataset):
    def __init__(self, data):
        self.inputSFP = data['InputSFP'].tolist()
        self.outputSFP = data['OutputSFP'].tolist()
        self.inputPPR = data['InputPPR'].tolist()
        self.outputPPR = data['OutputPPR'].tolist()
        self.inputHPPR = data['InputHPPR'].tolist()
        self.outputHPPR = data['OutputHPPR'].tolist()
       
       
    def __len__ (self):
        return len(self.outputPPR)
    
    def __getitem__(self, idx): 
        input1 = self.inputSFP[idx]
        input2 = self.inputPPR[idx]
        input3 = self.inputHPPR[idx]
        label1 = self.outputSFP[idx]
        label2 = self.outputPPR[idx]
        label3 = self.outputHPPR[idx]
        return input1,input2, input3, label1, label2, label3


In [ ]:
train_dataset = FantasyFootball(train)
valid_dataset = FantasyFootball(valid)
test_dataset = FantasyFootball(test)



In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [ ]:
inputSFP, inputPPR, inputHPPR, outputSFP, outputPPR, outputHPPR = next(iter(train_dataloader))

In [ ]:
inputSFP2 = []

for i in range (len(inputSFP)):
    inputSFP2.append(inputSFP[i].tolist())
    
inputSFP2 = np.array(inputSFP2)
inputSFP2 = torch.from_numpy(inputSFP2)
inputSFP2 = inputSFP2.double()

In [ ]:
#creating RNN model
class SportsModel(nn.Module): 
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(SportsModel, self).__init__()
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
       
        
        self.rnn = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
        self.float()
        
    def forward(self, x):
 
       
        batch_size = 64
        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        # Generates first hidden state, used in forwar propogation
        
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

input_size = 5
output_size = 1
hidden_dim = 128
n_layers = 1
    

In [ ]:
practice = SportsModel(input_size, output_size, hidden_dim, n_layers)
inputSFP2 = inputSFP2.reshape(64,5)
practice = practice.float()
out_inputSFP2,hidden_inputSFP2 = practice(inputSFP2.float())


In [ ]:
#loss funtions
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(practice.parameters(), lr = .01)

In [ ]:
#training the model
def get_preds_from_logits(logits):
    return out

epochs = 20 
train_losses = []
train_accuracies = []
for epoch in range(1, epochs+1):
    train_loss = [] 
    train_counts = 0
    
    
    practice.train()
    for train_inputSFP, train_inputPPR, train_inputHPPR, train_outputSFP, train_outputPPR, train_outputHPPR in train_dataloader:
        
       # train_inputSFP, train_outputSFP = train_inputSFP.to(device), train_outputSFP.to(device)
        train_inputSFP2 = []

        for i in range (len(train_inputSFP)):
            train_inputSFP2.append(train_inputSFP[i].tolist())
    
        train_inputSFP2 = np.array(train_inputSFP2)
        train_inputSFP2 = torch.from_numpy(train_inputSFP2)
        train_inputSFP2 = train_inputSFP2.float()
        
        
        m , n = train_inputSFP2.shape
       # print(m ,n)
       
        
        train_inputSFP2 = train_inputSFP2.reshape(n,5)
      #  print(train_inputSFP2.shape)
        optimizer.zero_grad()
        
        output, _ = practice(train_inputSFP2)
        output.reshape(n,1)
        #print(train_inputSFP2.type())
        loss = criterion(output, train_outputSFP.float())
        #print(output.type())
        #print(train_outputSFP.type())
        loss = loss.to(torch.float)
        
        loss.backward()
        
        optimizer.step()
        
       # SportsModel.eval()
        
        train_loss.append(loss.item())

    train_losses.append(sum(train_loss))    
    print(f'Epoch: {epoch} \tTraining Loss: {sum(train_loss)}')

In [ ]:
plt.plot(train_losses)
plt.show()